In [1]:
from tqdm import tqdm

In [2]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

The value specified in an AutoRun registry key could not be parsed.


The value specified in an AutoRun registry key could not be parsed.


In [3]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [4]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [5]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


**Step 1 : Create source embeddings for the text column**

In [6]:
from sentence_transformers import SentenceTransformer


e:\langchain_finance_news_research_tool_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [10]:
vectors.shape

(8, 768)

In [11]:
dim = vectors.shape[1]
dim

768

**Step 2 : Build a FAISS Index for vectors**

In [13]:
import faiss
index = faiss.IndexFlatL2(dim) # we use euclidian distance L2

**Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index**

In [14]:
index.add(vectors)

In [15]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000023481241B60> >

**Step 4 : Encode search text using same encorder and normalize the output vector**

In [ ]:
search_query = "I want to buy a polo t-shirt" 
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"

vec = encoder.encode(search_query)
vec.shape

(768,)

In [17]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [18]:
# faiss.normalize_L2(svec)

**Step 5: Search for similar vector in the FAISS index created**

In [20]:
distances , I = index.search(svec, k =2)
distances

array([[1.3844838, 1.4039094]], dtype=float32)

In [21]:
I

array([[3, 2]])

In [22]:
I.tolist()

[[3, 2]]

In [23]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [24]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [25]:
search_query

'I want to buy a polo t-shirt'

You can see that the two results from the dataframe are similar to a search_query